In [ ]:
# === Celda 1: Importar librerías ===
import numpy as np
import pandas as pd
import datetime
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn import metrics

from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap

import warnings
import sys

# Suprimir warnings y fijar semilla
if not sys.warnoptions:
    warnings.simplefilter("ignore")
np.random.seed(42)


In [ ]:
# === Celda 2: Descargar el dataset desde KaggleHub ===
import kagglehub

# Download latest version
path = kagglehub.dataset_download("imakash3011/customer-personality-analysis")

print("Path to dataset files:", path)


In [ ]:
# === Celda 3: Cargar el dataset ===
data = pd.read_csv('/home/cesar_unix/.cache/kagglehub/datasets/imakash3011/customer-personality-analysis/versions/1/marketing_campaign.csv', sep='\t')

data.head()

In [ ]:
data.info()

In [ ]:
# === Limpieza de datos ===
# Eliminar valores nulos
data = data.dropna()
len(data)

In [ ]:
#Modificar el formato de las fechas
data['Dt_Customer'] =pd.to_datetime(data['Dt_Customer'], format='mixed')
fechas = []
for i in data['Dt_Customer']:
    i = i.date()
    fechas.append(i)
dias = []
d1 = max(fechas)
for i in fechas:
    delta = d1 - i
    dias.append(delta)
data['Customer_For']=dias
data['Costumer_For'] = pd.to_numeric(data['Customer_For'], errors='coerce')


* Extraer la **Edad** de un cliente a partir del **Año_Nacimiento**, que indica el año de nacimiento de la persona  
* Crear otra característica **Gastado** que indique la cantidad total gastada por el cliente en varias categorías durante un período de dos años.  
* Crear otra característica **Viviendo_Con** a partir de **Estado_Civil** para extraer la situación de convivencia de las parejas.  
* Crear una característica **Hijos** para indicar el número total de hijos en un hogar, es decir, niños y adolescentes.  
* Para obtener más claridad sobre el hogar, crear una característica que indique el **Tamaño_Familiar**.  
* Crear una característica **Es_Padre** para indicar el estado de paternidad.  
* Eliminar algunas de las características redundantes.  


In [ ]:
# === Celda: Generación de nuevas características y limpieza ===
# Edad del cliente
data["Age"] = 2021 - data["Year_Birth"]

# Gasto total en los últimos dos años
data["spent"] = (
    data["MntWines"]
    + data["MntFruits"]
    + data["MntMeatProducts"]
    + data["MntFishProducts"]
    + data["MntSweetProducts"]
    + data["MntGoldProds"]
)

# Situación de convivencia
data["Living_with"] = data["Marital_Status"].replace({
    "Married": "Partner",
    "Together": "Partner",
    "Absurd": "Alone",
    "Widow": "Alone",
    "YOLO": "Alone",
    "Divorced": "Alone",
    "Single": "Alone",
})

# Número de hijos (niños + adolescentes)
data["Children"] = data["Kidhome"] + data["Teenhome"]

# Tamaño familiar: 1 si vive solo, 2 si vive en pareja, más hijos
data["Family_Size"] = (
    data["Living_with"].replace({"Alone": 1, "Partner": 2})
    + data["Children"]
)

# Indicador de paternidad/maternidad
data["Is_Parent"] = np.where(data["Children"] > 0, 1, 0)

# Simplificación de niveles educativos
data["Education"] = data["Education"].replace({
    "Basic": "Undergraduate",
    "2n Cycle": "Undergraduate",
    "Graduation": "Graduate",
    "Master": "Postgraduate",
    "PhD": "Postgraduate",
})

# Renombrar columnas para mayor claridad
data = data.rename(columns={
    "MntWines": "Wines",
    "MntFruits": "Fruits",
    "MntMeatProducts": "Meat",
    "MntFishProducts": "Fish",
    "MntSweetProducts": "Sweets",
    "MntGoldProds": "Gold"
})

# Eliminar columnas redundantes
to_drop = [
    "Marital_Status",
    "Dt_Customer",
    "Z_CostContact",
    "Z_Revenue",
    "Year_Birth",
    "ID"
]
data = data.drop(to_drop, axis=1)


In [ ]:
data.describe()

In [ ]:
# === Celda: Visualización de relaciones entre variables ===
Columnas = ["Income", "Recency", "Customer_For", "Age", "spent", "Is_Parent"]
plt.figure()
sns.pairplot(data[Columnas], hue="Is_Parent")
plt.show()


In [ ]:
# === Celda: Limpieza de datos y matriz de correlación ===
# Limpiar datos de edad e ingreso fuera de rango
data = data[data["Age"] < 90]
data = data[data["Income"] < 600000]

In [ ]:

# Matriz de correlación
corrmat = data.select_dtypes(include=np.number).corr()
plt.figure(figsize=(20, 20))
sns.heatmap(corrmat, annot=True, cmap=None, center=0)
plt.show()


In [ ]:
#Obtener  categorias que no son numéricas y pasarlas a numéricas
s = (data.dtypes == 'object')
object_cols = list(s[s].index)

print('Ctaergorías no numéricas:', object_cols)
LE= LabelEncoder()
#for i in object_cols:
 #   data[i] = LE.fit_transform(data[i])
for i in data.columns:
    data[i] = data[[i]].apply(LE.fit_transform)

In [ ]:
#Normalizar los datos
ds = data.copy()
cols_del = ['AcceptedCmp3','AcceptedCmp4','AcceptedCmp5','AcceptedCmp1','AcceptedCmp2','Complain','Response']
ds = ds.drop(cols_del, axis=1)
scaler = StandardScaler()
scaler.fit(ds)
scaled_ds = pd.DataFrame(scaler.transform(ds), columns=ds.columns)


In [ ]:
scaled_ds.head()

In [ ]:
#Usar PCA para reducir la dimensionalidad a 3 dimensiones
pca = PCA(n_components=3)
pca.fit(scaled_ds)
PCA_ds = pd.DataFrame(pca.transform(scaled_ds), columns=['col1', 'col2', 'col3'])
PCA_ds.describe().T

In [ ]:
X = PCA_ds['col1']
Y = PCA_ds['col2']
Z = PCA_ds['col3']
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X, Y, Z, c='blue', marker='o')
plt.show()

In [ ]:
#Metodo del codo para dterminar el número de clusters
Elbow_M = KElbowVisualizer(KMeans(), k=(1, 10))
Elbow_M.fit(PCA_ds)
Elbow_M.show()

In [ ]:
#Crear el numero de clusters
AC = AgglomerativeClustering(n_clusters=4)
yhat_AC = AC.fit_predict(PCA_ds)
PCA_ds['Cluster'] = yhat_AC
data['Cluster'] = yhat_AC

In [ ]:
colors = ['red', 'green', 'blue', 'yellow']
cmap = ListedColormap(colors)
fig = plt.figure(figsize=(10, 8))
ax = plt.subplot(111, projection='3d', label='bla')
ax.scatter(X,Y,Z, s = 40, c = PCA_ds['Cluster'], marker='o', cmap=cmap)
plt.show()

In [ ]:
#Evaluar los datos
pl = sns.countplot(x=data['Cluster'], palette=colors)
plt.title('Distribución de los clusters')
plt.show()

In [ ]:
pl=sns.scatterplot(data = data, x=data["spent"], y=data["Income"], hue=data["Cluster"], palette=colors)
plt.legend()
plt.show()


* grupo 0: gastos medio e ingresos medios
* grupo 1: gastos altos e ingresos altos
* grupo 2: gastos bajos e ingresos bajos
* grupo 3: gastos medio e ingresos bajos

In [ ]:
plt.figure()
pl = sns.boxplot(x=data['Cluster'], y=data['spent'], palette=colors)
plt.show()

In [ ]:
data['Total_Promos'] = data['AcceptedCmp1'] + data['AcceptedCmp2'] + data['AcceptedCmp3'] + data['AcceptedCmp4'] + data['AcceptedCmp5']
plt.figure()
pl = sns.countplot(x=data['Total_Promos'], hue=data['Cluster'], palette=colors)
pl.set_xlabel('Numero de promociones aceptadas')
plt.show()

In [ ]:
#mostrar el numero de ofertas
plt.figure()
pl = sns.boxplot(y=data['NumDealsPurchases'], x =data['Cluster'], palette=colors)
plt.show()